In [37]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

In [38]:
model = load_model('ann_model.h5')

with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)

with open('onehot_encoder_geo.pkl', 'rb') as f:
    onehot_encoder_geo = pickle.load(f)

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

In [39]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [40]:
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\navee\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [41]:
input_df = pd.DataFrame([input_data])
input_df = pd.concat([input_df, geo_encoded_df], axis=1)

In [42]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [43]:
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df = input_df.drop(columns=['Geography'])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [44]:
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [45]:
prediction = model.predict(input_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step


In [46]:
prediction

array([[0.03017315]], dtype=float32)

In [48]:
prediction_proba = prediction[0][0]
prediction_proba

0.030173147

In [49]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.


In [50]:
onehot_encoder_geo.categories_

[array(['France', 'Germany', 'Spain'], dtype=object)]

In [51]:
label_encoder_gender.classes_

array(['Female', 'Male'], dtype=object)